## Ripple stats during sleep deprivation

In [37]:
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd
from ModulesPath.plotUtil import Fig
import subjects
import seaborn as sns

sessions = subjects.Sd().allsess + subjects.Sd().utkuAG_day1 + subjects.Sd().utkuAG_day2
# sessions = subjects.Nsd().allsess
print(sessions)

[processData(RatJDay1), processData(RatKDay1), processData(RatNDay1), processData(RatSDay3SD), processData(RatRDay2SD), processData(UtkuAG_2019-12-22_SD_day1), processData(UtkuAG_2019-12-26_SD_day2)]


In [39]:
%matplotlib widget
figure = Fig()
fig,gs = figure.draw(grid=(1,1),size=(4,4),style='Pres')
df = pd.DataFrame()
for sub,sess in enumerate(sessions):

    post = sess.paradigm.post
    # pre = sess.paradigm.pre
    # sd_period = [post[0],post[1]]
    # sd_period = [post[0],post[0]+9*3600]
    try:
        sd_period = sess.paradigm.sd
    except:
        sd_period = [sess.paradigm.post[0],sess.paradigm.post[0]+5*3600]
    rpls = sess.ripple.epochs
    # states = sess.brainstates.epochs

    binsz = 600
    sd_bin = np.arange(sd_period[0],sd_period[1]+binsz,binsz)
    hist_rpl = np.histogram(rpls.start,bins=sd_bin)[0]

    nrem_percent = []
    for start in sd_bin[:-1]:
        prop = sess.brainstates.proportion([start,start+binsz])
        nrem_percent.append(prop['nrem'])

    df = df.append(pd.DataFrame({'n_ripples':hist_rpl/binsz,'tbin':(sd_bin[:-1]-sd_bin[0])/3600,'nrem':nrem_percent,'sub':sub}))


ax = plt.subplot(gs[0])
rpls_mean = df.groupby('tbin').mean().n_ripples
rpls_sem = df.groupby('tbin').sem(ddof=0).n_ripples
ax.fill_between(rpls_mean.index,rpls_mean-rpls_sem,rpls_mean+rpls_sem,color='#faa49d',alpha=0.7)
ax.plot(rpls_mean.index,rpls_mean,color='#faa49d',zorder=2)
ax.axvline(5,ls='--',color='r')
ax.set_xlabel('ZT time (h)')
ax.set_ylabel('Ripple rate')


axsleep= ax.twinx()
color = sess.brainstates.colors['nrem']
sleep_df = df.groupby('tbin').mean()
axsleep.fill_between(sleep_df.index,sleep_df.nrem,color=color,zorder=1,alpha=0.3)
# sns.lineplot(data=df,x='tbin',y='nrem',ax=axsleep,estimator='sum')
axsleep.set_ylim([0,1])
axsleep.spines['right'].set_visible(True)
axsleep.set_ylabel('nrem proportion')



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'nrem proportion')

In [40]:
sess.ripple.epochs

,start,stop,peakpower,peaktime,duration,label
0,45.8168,46.2432,0.004161,46.0792,0.4264,
1,110.9728,111.3568,0.004092,111.0112,0.3840,
2,111.7744,112.1136,0.004712,111.8224,0.3392,
3,129.7888,129.9936,0.004017,129.9344,0.2048,
4,163.9040,164.3088,0.004323,164.0112,0.4048,
...,...,...,...,...,...,...
10806,46378.5456,46378.6368,0.005146,46378.6296,0.0912,
10807,46379.0264,46379.2360,0.008585,46379.1288,0.2096,
10808,46379.3320,46379.4360,0.007473,46379.3568,0.1040,
10809,46381.6128,46381.6784,0.004298,46381.6352,0.0656,
